# Helpers

In [1]:
import re
from collections import Counter, defaultdict
import itertools

In [68]:
# A lot of this code is inspired from Peter Norvig's style.

def id(x): return x

def mapt(fun, it): return tuple(map(fun, it))
def first(it, pred): return next(filter(pred, it))
def parse_uint(line): return mapt(int, re.findall(r'\d+', line))
def  parse_int(line): return mapt(int, re.findall(r'-?\d+', line))

        
def read(day, process=id, split='\n'):
    with open(f'./inputs/{day}.txt') as f:
        return mapt(process, f.read().split(split))
    
def partition_into(lst: list, when, preprocess=id, postprocess=id) -> list[list]:
    res = []
    for item in map(preprocess, lst):
        if when(item): 
            yield res
            res = []
        else: 
            res.append(postprocess(item))

def partition(inp, segment_length):
    for i in range(0, len(inp), segment_length):
        yield inp[i:i+segment_length]

def windows(inp, n):
    for i in range(len(inp)):
        yield inp[i:i+n]

# Day 1

In [43]:
input = partition_into(read(1), lambda x: not x, postprocess=int)
calories = [sum(l) for  l in input]
calories.sort()
assert calories[-1] == 66487
assert sum(calories[-3:]) == 197301

# Day 2

In [4]:
scores = {
    'A X': 1+3,
    'A Y': 2+6,
    'A Z': 3+0,
    'B X': 1+0,
    'B Y': 2+3,
    'B Z': 3+6,
    'C X': 1+6,
    'C Y': 2+0,
    'C Z': 3+3,
}

In [44]:
input = read(2)[:-1]

In [45]:
assert sum(map(scores.get, input)) == 10994

In [46]:
scores2 = {
    'A X': 3+0,
    'A Y': 1+3,
    'A Z': 2+6,
    'B X': 1+0,
    'B Y': 2+3,
    'B Z': 3+6,
    'C X': 2+0,
    'C Y': 3+3,
    'C Z': 1+6,
}

In [47]:
assert sum(map(scores2.get, input)) == 12526

# Day 3

In [48]:
import string

priority = dict(enumerate(string.ascii_lowercase, 1))
priority |= dict(enumerate(string.ascii_uppercase, 27))
priority = {v:k for k,v in priority.items()}

In [49]:
input = read(3, str.strip)[:-1]

In [50]:
def split_half (x):
    n = len(x)//2
    return set(x[:n]), set(x[n:])

common = [x&y for x, y in map(split_half, input)]

In [51]:
sum(sum(map(priority.get, c)) for c in common)

7848

In [52]:
groups = [input[i:i+3] for i in range(0, len(input), 3)]
groups = [list(set(x)&set(y)&set(z))[0] for x, y, z in groups]
sum(map(priority.get, groups))

2616

# Day 4

In [53]:
input = read(4, parse_uint)[:-1]

In [54]:
def overlap(args):
    a, b, c, d = args
    return a <= c <= d <= b or c <= a <= b <= d

In [55]:
assert sum(map(overlap, input)) == 582

In [56]:
def partial(args):
    a, b, c, d = args
    return a <= c <= b <= d or c <= a <= d <= b or overlap(args)

In [57]:
assert sum(map(partial, input)) == 893

# Day 5

In [58]:
input = read(5, lambda x: f' {x}')[:-1]

In [59]:
crates = [[] for _ in range(10)]
for i in range(8):
    for j, elem in enumerate(partition(input[i], 4), 1):
        if elem.strip():
            crates[j].append(elem[2])

crates_orig = [crate[::-1] for crate in crates]

In [60]:
crates = [crate.copy() for crate in crates_orig]
for line in input[10:]:
    a, b, c = parse_int(line)
    for j in range(a):
        crates[c].append(crates[b].pop())

In [61]:
assert ''.join(crate[-1] for crate in crates[1:]) == 'CWMTGHBDW'

In [62]:
crates = [crate.copy() for crate in crates_orig]
for line in input[10:]:
    a, b, c = parse_int(line)
    crates[c].extend(crates[b][-a:])
    crates[b] = crates[b][:-a]

In [63]:
assert ''.join(crate[-1] for crate in crates[1:]) == 'SSCGWJCRB'

## Day 6

In [64]:
input = read(6)[0]

In [65]:
def solve_6(input, L):
    for i, window in enumerate(windows(input, L)):
        if len(set(window)) == L:
            return i+L

In [66]:
assert solve_6(input, 4) == 1480
assert solve_6(input, 14) == 2746

# Day 7

In [311]:
# Note: This is not the first version of the code I wrote.
# The first version was messy, and is located inside `original.py`.
# It simulated the whole filesystem as a tree, and did a depth-first-search.

In [312]:
input = read(7, split='$')[1:]

In [313]:
cwd = []
dirs = Counter()
for command in input:
    command, *output = command.strip().split()
    match command:
        case 'cd':
            match output[0]:
                case '/': 
                    cwd = ['/']
                case '..': 
                    if cwd: cwd.pop()
                case d:
                    cwd.append(d)
        case 'ls':
            if '/'.join(cwd) not in dirs:
                s = sum(int(a) for a, b in partition(output, 2) if a != 'dir')
                for i in range(len(cwd)):
                    dirs['/'.join(cwd[:i+1])] += s


In [314]:
assert sum(v for v in dirs.values() if v <= 100000) == 1306611

In [315]:
need = 30000000 - (70000000-dirs['/'])
assert min(v for v in dirs.values() if v >= need) == 13210366

# Day 8

In [326]:
import numpy as np

In [418]:
input = read(8)[:-1]
# input = '''30373
# 25512
# 65332
# 33549
# 35390'''.split()


In [419]:
input = [mapt(int, line) for line in input]

In [404]:
grid = np.array(input)
h, w = grid.shape

In [381]:
td = np.maximum.accumulate(grid, axis=0)
lr = np.maximum.accumulate(grid, axis=1)
dt = np.maximum.accumulate(grid[::-1, :], axis=0)[::-1, :]
rl = np.maximum.accumulate(grid[:, ::-1], axis=1)[:, ::-1]

In [382]:
td = td[0:-2, 1:-1]
lr = lr[1:-1, 0:-2]
dt = dt[2:, 1:-1]
rl = rl[1:-1, 2:]

In [383]:
grid = grid[1:-1, 1:-1]

In [384]:
flag = grid > td
flag |= grid > dt
flag |= grid > lr
flag |= grid > rl

In [385]:
flag.sum() + h*2 + (w-2)*2

1690

In [420]:
grid = np.array(input)
h, w = grid.shape

In [421]:
mx = 0
for i in range(h):
    for j in range(w):
        col = (grid[:, j] - grid[i, j]) >= 0
        
        try:
            t = col[:i][::-1].tolist().index(True)+1
        except: 
            t = i
            
        try:
            b = col[i+1:].tolist().index(True)+1
        except:
            b = h-i-1
        
        
        row = (grid[i, :] - grid[i, j]) >= 0
        try:
            l = row[:j][::-1].tolist().index(True)+1
        except:
            l = j
            
        try:
            r = row[j+1:].tolist().index(True)+1
        except:
            r = w-j-1
        
        mx = max(mx, t*b*l*r)

In [422]:
mx

535680

In [403]:
w

99

# Day 9

In [471]:
input = '''R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2'''.split()

input = '''R 5
U 8
L 8
D 3
R 17
D 10
L 25
U 20'''.split()

In [464]:
input = read(9, split=None)

In [465]:
hr = hc = tr = tc = 0

visited = {(0, 0)}
for command in partition(input, 2):
    match command:
        case ('U', x):
            for _ in range(int(x)):
                hr += 1
                if tr < hr-1:
                    tc = hc
                    tr = hr-1
                    visited.add((tr, tc))
                # print(command, tr, tc)
        case ('D', x):
            for _ in range(int(x)):
                hr -= 1
                if tr > hr+1:
                    tc = hc
                    tr = hr+1
                    visited.add((tr, tc))
                # print(command, tr, tc)
        case ('L', x):
            for _ in range(int(x)):
                hc -= 1
                if tc > hc+1:
                    tr = hr
                    tc = hc+1
                    visited.add((tr, tc))
                # print(command, tr, tc)
        case ('R', x):
            for _ in range(int(x)):
                hc += 1
                if tc < hc-1:
                    tr = hr
                    tc = hc-1
                    visited.add((tr, tc))
                # print(command, tr, tc)


In [506]:
def vis_grid(pos):
    min_r = min(p[0] for p in pos)
    min_c = min(p[1] for p in pos)
    
    width = 50
    height = 50
    
    grid = [['.' for _ in range(width)] 
            for _ in range(height)]
    for i, (r, c) in enumerate(pos[::-1]):
        grid[min_r+r][min_c+c] = chr(ord('9') - i)
    
    return '\n'.join(''.join(line) for line in grid[::-1])

In [510]:
pos

[(17, 0),
 (25, 0),
 (33, 0),
 (41, 0),
 (49, 0),
 (57, 0),
 (65, 0),
 (64, 0),
 [0, 0],
 (9, 0)]

In [516]:
pos = [[0,0] for i in range(10)]

visited = {(0, 0)}
for (command, repeat) in partition(input, 2):
    print('X'*40)
    print(vis_grid(pos))
    # print(pos)
    for i in range(int(repeat)):
        hr, hc = pos[0]
        for j in range(1, 10):
            tr, tc = pos[j]
            moved = False
            match command:
                case 'U':
                    hr += 1
                    if tr < hr-1:
                        tc = hc
                        tr = hr-1
                        moved = True
                case 'D':
                    hr -= 1
                    if tr > hr+1:
                        tc = hc
                        tr = hr+1
                        moved = True
                case 'L':
                    hc -= 1
                    if tc > hc+1:
                        tr = hr
                        tc = hc+1
                        moved = True
                case 'R':
                    hc += 1
                    if tc < hc-1:
                        tr = hr
                        tc = hc-1
                        moved = True
            pos[j-1] = hr, hc
            hr, hc = tr, tc
            if not moved: break
        visited.add((tr, tc))
        pos[9] = (tr, tc)

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
.........................................

In [502]:

min_r = min(p[0] for p in pos)
min_c = min(p[1] for p in pos)
width = 30
height = 30

grid = [['.' for _ in range(width)] 
        for _ in range(height)]
for i, (r, c) in enumerate(pos[::-1]):
    grid[min_r+r][min_c+c] = chr(ord('9') - i)

In [500]:
print(vis_grid(pos))

..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
............................9.....................
............................8.....................
............................7.....................
............................6.....................
............................5.....................
............................4.....................
............................3.....................
............................2..

# Day 10

In [626]:
input = read(10, split='\n')[:-1]

In [631]:
check_at = 20
cycle = 1
X = 1
part1 = 0
for inst in input:
    if cycle <= check_at <= cycle+1:
        part1 += X*check_at
        check_at += 40
    
    if inst == 'noop':
        cycle += 1
    else:
        cycle += 2
        X += int(inst.split()[1])
print(part1)

11220


In [632]:
C = 0
X = 1
output = []
row = []

def draw():
    global row        
    c = C%40 
    row.append('#'  if X-1 <= c <= X+1 else '.')
    if len(row) == 40:
        output.append(row)
        row = []
   
for inst in input:
    if inst == 'noop':
        draw()
        C += 1
    else:
        draw()
        C += 1
        draw()
        C += 1
        X += int(inst.split()[1])

In [633]:
print('\n'.join(''.join(line) for line in output))

###..####.###...##....##.####.#....#..#.
#..#....#.#..#.#..#....#.#....#....#.#..
###....#..#..#.#..#....#.###..#....##...
#..#..#...###..####....#.#....#....#.#..
#..#.#....#....#..#.#..#.#....#....#.#..
###..####.#....#..#..##..####.####.#..#.
